<a href="https://colab.research.google.com/github/KushangShah/Image-Classification-using-ML/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
print("Hello, World!")

Hello, World!
